In [ ]:
# 000.UTILIDADES.ipynb
# %% [markdown]
"""
# 000.UTILIDADES: Copiar Arquivos entre Pastas no Google Drive

Este notebook permite copiar todo o conteúdo de uma pasta de origem para uma pasta de destino
direto no Google Drive, usando o rclone. Basta fornecer os IDs das pastas.
"""
# %% [code]
import subprocess

# %% [code]
# Cabeçalho
print("--- Copiar Arquivos no Google Drive ---")

# %% [code]
# Ler os IDs das pastas do usuário
title = "--- Copiar Arquivos no Google Drive ---"
print(title)
origin_id = input("ID_DA_PASTA_ORIGEM: ").strip()
dest_id   = input("ID_DA_PASTA_DESTINO: ").strip()

# %% [code]
# Construir os caminhos remotos
origin_path = f"gdrive:{origin_id}"
dest_path   = f"gdrive:{dest_id}"
print(f"Origem: {origin_path}")
print(f"Destino: {dest_path}")

# %% [code]
# Executar a cópia via rclone
print("Iniciando cópia...")
result = subprocess.run(
    [
        "rclone", "copy", origin_path, dest_path,
        "--create-empty-src-dirs"
    ],
    capture_output=True,
    text=True
)
if result.returncode == 0:
    print("✓ Copia realizada com sucesso!")
else:
    print(f"✗ Erro na cópia (code {result.returncode}):")
    print(result.stderr)

# %% [code]
# Listar conteúdo da pasta de destino para verificação
print("\nConteúdo da pasta de destino:")
subprocess.run(["rclone", "ls", dest_path])

